# Test out Firestore DB with opencv tracking

In [1]:
import cv2, firebase_admin, time
from firebase_admin import credentials
from firebase_admin import firestore

## Initialize Firestore instance and input credential file

In [2]:
cred = credentials.Certificate(jsoncred)
firebase_admin.initialize_app(cred)

db = firestore.client()

## - test with CSRT tracker

In [3]:
tracker = cv2.TrackerCSRT_create()

In [4]:
v = cv2.VideoCapture(r'walk2.mp4')

cameraID = 'campus-harbor-1'

In [5]:
i=0
ret, frame = v.read()

frameID='frame-number-'+str(i)
timestamp = time.time()
begintime=timestamp

cv2.namedWindow('Frame', cv2.WINDOW_NORMAL)
cv2.imshow('Frame',frame)
fps = v.get(cv2.CAP_PROP_FPS)

## Create a database collection for the current camera (or video) and create a document with cam info

In [6]:
camera_ref = db.collection(cameraID).document('cameraInfo')

camera_ref.set({
    
    'location':'at the harbor',
    'ID':cameraID,
    'FrameRate':fps,
    'IP Address': '10.0.0.1'
})

objectID = 'selectedObject-1'

## - define a function to collect specified infomation of tracked objects on every frame

In [7]:
def setFirestoreDB(cameraID, frameID, timeID, objectID, boundingBox):
    frame_ref = db.collection(cameraID).document(frameID)
    frame_ref.set({
        'objectID':objectID,
        'boundingBox':boundingBox,
        'timeStamp':timeID
    })

## -* ensure to select an object or ROI to be tracked from the first frame

In [8]:
bb = cv2.selectROI('Frame',frame)
tracker.init(frame,bb)

True

In [9]:
setFirestoreDB(cameraID, frameID, timestamp, objectID, bb)

In [10]:
while True:
    ret, frame = v.read()
    
    i+=1
    frameID='frame-number-'+str(i)
    timestamp = time.time()
        
    if not ret:
        break
    (success,box) = tracker.update(frame)
    
    setFirestoreDB(cameraID, frameID, timestamp, objectID, box)
    
    if success:
        (x,y,w,h)=[int(a) for a in box]
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
    cv2.imshow('Frame',frame)
    key = cv2.waitKey(5) & 0xFF
    if key == ord('q'):
        break
v.release()
cv2.destroyAllWindows()

# Sample Query of the database collection to perform specific tasks

In [11]:
docs = db.collection(cameraID).where('timeStamp','>=',begintime+50).where('timeStamp','<',begintime+51).stream()


for doc in docs:
    print('{} => {}'.format(doc.id, doc.to_dict()))

frame-number-176 => {'boundingBox': [-29.529827117919922, 194.1931915283203, 75.47994232177734, 210.11984252929688], 'timeStamp': 1607467688.2888384, 'objectID': 'selectedObject-1'}
frame-number-177 => {'timeStamp': 1607467688.479769, 'boundingBox': [-29.657026290893555, 194.06227111816406, 75.47994232177734, 210.11984252929688], 'objectID': 'selectedObject-1'}
frame-number-178 => {'objectID': 'selectedObject-1', 'timeStamp': 1607467688.6651442, 'boundingBox': [-29.597820281982422, 193.90892028808594, 75.47994232177734, 210.11984252929688]}
frame-number-179 => {'objectID': 'selectedObject-1', 'boundingBox': [-29.326187133789062, 194.32249450683594, 75.47994232177734, 210.11984252929688], 'timeStamp': 1607467688.8613057}
frame-number-180 => {'timeStamp': 1607467689.0411499, 'objectID': 'selectedObject-1', 'boundingBox': [-28.787864685058594, 194.1807098388672, 75.47994232177734, 210.11984252929688]}
